In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output

In [3]:
s = Service('chromedriver')
driver = webdriver.Chrome(service = s)

In [4]:

url = "https://www.airbnb.com.sg/?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&search_mode=flex_destinations_search&flexible_trip_lengths%5B%5D=one_week&location_search=MIN_MAP_BOUNDS&price_filter_input_type=0&category_tag=Tag%3A7769&search_type=category_change&_set_bev_on_new_domain=1663217624_YWUwOTgyNDg3MjQ4"
driver.get(url)


In [5]:
product_list = driver.find_elements(by = By.CLASS_NAME, value ="ln2bl2p")
product_list

[<selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="b70ff529-dad9-47c9-92b8-feaf98f72c2c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="46275866-1159-4eba-8724-2b9cf7c193ce")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="7dbd733d-b0bf-4016-84e4-c5fe568b2995")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="5126fe98-398f-4db6-bd3e-3853a1c55caf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="1040f7ce-d91e-44f4-be28-1b0295e85418")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="33c35f24-0266-484e-a2b5-ea753dcb2de7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d9ad879912a7306462fbe1edde5a116", element="11d5f959-b3ef-4e21-98f0-5e

In [6]:
url_list = [p.get_attribute('href') for p in product_list]
url_list


['https://www.airbnb.com.sg/rooms/34345351?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-09-16&check_out=2022-09-22&previous_page_section_name=1000&federated_search_id=e3fb08f2-9f4d-485b-b6ef-9358841d0f1a',
 'https://www.airbnb.com.sg/rooms/53871587?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-09-18&check_out=2022-09-23&previous_page_section_name=1000&federated_search_id=e3fb08f2-9f4d-485b-b6ef-9358841d0f1a',
 'https://www.airbnb.com.sg/rooms/31665667?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-10-16&check_out=2022-10-21&previous_page_section_name=1000&federated_search_id=e3fb08f2-9f4d-485b-b6ef-9358841d0f1a',
 'https://www.airbnb.com.sg/rooms/47463008?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-09-25&check_out=2022-09-30&previous_page_section_name=

In [7]:
with open("Product_URL_List.txt", "w") as f:
    for i in url_list:
        f.write(f"{i}")
        f.write('\n')

In [8]:
def open_driver():
    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    s = Service('chromedriver')
    driver = webdriver.Chrome(service=s, options=op)
    return driver

In [9]:
def headline_scrap(driver):
    try:
        headline = driver.find_element(by=By.TAG_NAME, value = 'h1').text
        print("Title: ",headline)
        return headline
    except:
        print("Headline is not found")
        return 'N/A'

In [10]:
def rating_scrap(driver):
    try:
        rating = driver.find_element(by=By.CLASS_NAME, value = '_1jvg42j').text
        print("Rating: ",rating)
        return rating
    except:
        print("No Rating")
        return 'N/A'

In [11]:
def address_scrap(driver):
    try:
        address = driver.find_element(by=By.CLASS_NAME, value = '_9xiloll').text
        print("Addresss: ",address)
        return address
    except:
        print("Address is no found")
        return 'N/A'

In [12]:
def image_scrap(driver, category):
    try:
        images = driver.find_element(by=By.XPATH, value = '//*[@id="FMP-target"]')
        img_path = []
        for img in images.find_elements(by=By.TAG_NAME, value = 'img'):
            img_urls = img.get_attribute('src')

            img_urls_rename = img_urls + '/620/466/fitted.jpg'

            #img_name = '@'.join(img_urls_firstname.split('/')[-2:])

            r = requests.get(img_urls_rename)
            i = Image.open(BytesIO(r.content))
            i.save(f"img/{category}/{img_name}.jpg")
            img_path.append(img_name)
            return img_path
    except:
        print("No Image")
        return "N/A"
    

In [13]:
def price_scrap(driver):
    try:
        price = driver.find_element(by=By.XPATH, value = '/html/body/div[5]/div/div/div[1]/div/div/div[1]/div/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]').text
        print("Price: ",price)
        return price
    except:
        print("Price is not given")
        return 'N/A'

In [14]:
def description_scraping(driver):
    try:
        description = driver.find_element(by=By.CLASS_NAME, value = 'll4r2nl').text
        print("Description: ",description)
        return description
    except:
        print("No Description")
        return 'N/A'

In [15]:
def place_offer_scraping(driver):
    try:
        place_offer = driver.find_element(by=By.XPATH, value = '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[6]/div/div[2]/section').text
        print("What this place offers: ",place_offer)
        return place_offer
    except:
        print("No description of what it offers")
        return "N/A"

In [16]:
def all_review_scraping(driver):
    try:
        review = driver.find_element(by=By.XPATH, value = '//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section').text
        print("Review: ",review)
        return review
    except:
        print("There is no review")
        return 'N/A'

In [17]:
def product_scrap(driver, url, category):
    driver.get(url)
    time.sleep(3)
    #scraping headlines
    headline = headline_scrap(driver)
#     if not headline:
#         return [None]*8
    
    #rating scraping
    rating = rating_scrap(driver)
    
    #address scraping
    address = address_scrap(driver)
    
    #image_scraping
    image = image_scrap(driver, category)
    
    #price scraping
    price = price_scrap(driver)
    
    #description_scraping
    description = description_scraping(driver)
    
    #what this place offers scraping
    place_offer = place_offer_scraping(driver)
    
    #Review scraping
    all_review = all_review_scraping(driver)
    
    return headline, rating, address, image, price, description, place_offer,all_review

In [18]:
category = 'Beaches'
with open('Product_URL_List.txt', 'r') as f:
    single_url = f.readlines()
    headlines, ratings, addresses, images, prices, descriptions, what_this_place_offers,  reviews = [], [], [], [], [], [], [], []
    cnt = 1
#     print(single_url)
    for s in single_url:
        print(cnt, s)
        driver = open_driver()
        
        headline, rating, address, image, price, description, place_offer,  all_review = product_scrap(driver, s, category)
        #if headline:
        headlines.append(headline)
        ratings.append(rating)
        addresses.append(address)
        images.append(image)
        prices.append(price)
        descriptions.append(description)
        what_this_place_offers.append(place_offer)
        reviews.append(all_review)


        cnt += 1
        print("=============================================")
        clear_output(wait=True)

20 https://www.airbnb.com.sg/rooms/16948499?adults=1&category_tag=Tag%3A7769&children=0&infants=0&search_mode=flex_destinations_search&check_in=2022-09-26&check_out=2022-10-01&previous_page_section_name=1000&federated_search_id=e3fb08f2-9f4d-485b-b6ef-9358841d0f1a

Title:  The Nipah Chalet Beach Front 1
Rating:  5.0 ·
59 reviews
Addresss:  Tioman Island, Mersing, Malaysia
Price:  $78 SGD
Description:  More or less 8 steps to sea
No description of what it offers
Review:  5.0 out of 5 stars from 59 reviews
5.0 · 59 reviews
Cleanliness
4.8
Accuracy
4.9
Communication
4.9
Location
5.0
Check-in
5.0
Value
4.8
Gaurav
August 2022
Abbas is gem of a person and a wonderful host. Location is awesome. Come here if you want to get disconnected from the world. Lovely cats on the property. And a cute ball of fur LuLu was our fav.
Abbas prepare finger licking food.
All in all, give it a try and you wouldn’t regret.
Show more
Nicholas
September 2020
This place is a GEM. My friends had such a great time h

In [24]:
with open("airnbn_beaches.txt", "w", encoding='utf-8') as f:
    for h, ra, ad, img, pr, des, po, re in zip(headlines, ratings, addresses, images, prices, descriptions, what_this_place_offers,  reviews):
        f.write(f"{h}, {ra}, {ad}, {img}, {pr}, {des}, {po}, {re}")
        f.write("\n")
        f.write("==========================================================================================================")